This notebook will figure out how to correctly format an API call to the visual crossing weather app. Documentation is here: 
>https://www.visualcrossing.com/resources/documentation/weather-api/timeline-weather-api/

The goal is to have a function that can be given the lat/long and date/time of an event and return the weather conditions.

In [4]:
# get API tokens
with open('.env.development') as f:
    data = f.read()

for line in data.split('\n'):
    head,sep,tail = line.partition(' = ')
    
    if head == 'crashAPIkey':
        crashAPIkey = tail
    elif head == 'weatherAPIkey':
        weatherAPIkey = tail

#print(f'CrashAPIToken {crashAPIkey},WeatherAPIToken {weatherAPIkey} ')

In [5]:
lat = '39.711714'
lon = '-90.191958'
date = '2020-12-15'
time = '14:00:00'

In [ ]:
ApiQuery = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'+\
            lat+'%2C'+lon+'/'+date+'T'+time+'?unitGroup=us&key='+weatherAPIkey+'&include=current&contentType=json'

#print(ApiQuery)

In [7]:
import urllib
try: 
    with urllib.request.urlopen(ApiQuery) as response:
        html = response.read()
except urllib.error.HTTPError  as e:
    ErrorInfo= e.read().decode() 
    print('Error code: ', e.code, ErrorInfo)
    sys.exit()
except  urllib.error.URLError as e:
    ErrorInfo= e.read().decode() 
    print('Error code: ', e.code,ErrorInfo)
    sys.exit()

In [8]:
import json
weather = json.loads(html.decode('utf-8'))

In [9]:
weather

{'queryCost': 1,
 'latitude': 39.711714,
 'longitude': -90.191958,
 'resolvedAddress': '39.711714,-90.191958',
 'address': '39.711714,-90.191958',
 'timezone': 'America/Chicago',
 'tzoffset': -6.0,
 'days': [{'datetime': '2020-12-15',
   'datetimeEpoch': 1608012000,
   'tempmax': 31.7,
   'tempmin': 20.7,
   'temp': 26.0,
   'feelslikemax': 24.1,
   'feelslikemin': 11.4,
   'feelslike': 17.3,
   'dew': 22.4,
   'humidity': 86.1,
   'precip': 0.0,
   'precipprob': 0.0,
   'precipcover': 0.0,
   'preciptype': None,
   'snow': 0.0,
   'snowdepth': 0.0,
   'windgust': 25.3,
   'windspeed': 15.9,
   'winddir': 57.3,
   'pressure': 1025.1,
   'cloudcover': 29.8,
   'visibility': 9.9,
   'solarradiation': 61.6,
   'solarenergy': 3.3,
   'uvindex': 2.0,
   'sunrise': '07:15:05',
   'sunriseEpoch': 1608038105,
   'sunset': '16:37:23',
   'sunsetEpoch': 1608071843,
   'moonphase': 0.0,
   'conditions': 'Partially cloudy',
   'description': 'Becoming cloudy in the afternoon.',
   'icon': 'partly-

In [10]:
type(weather)

dict

`weather` is a dictionary with a number of K,V, including:
* `days`, a one-entry list containing a dictionary of values for the given day such as
    * datetime
    * tempmax / tempmin / temp (average?)
    * feelslikemax / feelslikemin / feelslike
    * dew / humidity
    * precip / precipprob / preciptype
    * snow / snowdepth
    * windgust / windspeed / winddir
    * sunrise / sunset
* `stations`, a dictionary of dictionary values for the stations providing data
* `currentConditions`, a dictionary of values for the date and time specified, including:
    * temp / feelslike
    * dew / humidity
    * precip / precipprob / preciptype
    * snow / snowdepth
    * windgust / windspeed / winddir
    * conditions

days and current conditions can be accessed like this:
> tempmax = weather['days'][0]['tempmax']

> temp = weather['currentConditions']['temp']

In [23]:
for item in ['temp','windspeed','winddir','preciptype','conditions']:
    val = weather['currentConditions'][item]
    print(f'{item} reported to be {val}')

temp reported to be 30.9
windspeed reported to be 10.2
winddir reported to be 60.0
preciptype reported to be None
conditions reported to be Overcast


How to add these to a DF?
1) Define a function that takes the relevant values from the DF and returns whatever info is desired from HTML lookup

2) Add it to the DF

In [3]:
def weatherLookup(lat,lon,dateTime,APIkey=weatherAPIkey):
    import urllib
    import json
    # the data in the DF is float and dateTime, which needs to be convered to strings to be added to the API query
    latS, lonS  = str(lat), str(lon)
    dateTimeS = dateTime.strftime('%Y-%m-%dT%H:%M:%S') #YYYY-MM-DDTHH:MM:SS
    ApiQuery = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'+\
            latS+'%2C'+lonS+'/'+dateTimeS+'?unitGroup=us&key='+APIkey+'&include=current&contentType=json'
    try: 
        with urllib.request.urlopen(ApiQuery) as response:
            html = response.read()
    except urllib.error.HTTPError  as e:
        ErrorInfo= e.read().decode() 
        print('Error code: ', e.code, ErrorInfo)
        sys.exit()
    except  urllib.error.URLError as e:
        ErrorInfo= e.read().decode() 
        print('Error code: ', e.code,ErrorInfo)
        sys.exit()

    weather = json.loads(html.decode('utf-8'))
    return weather['currentConditions']['temp'], weather['currentConditions']['conditions'], weather['currentConditions']['preciptype']

In [ ]:
# add three columns to the DF with the values returned from the weatherLookup function
test[['wTemp','wCond','wPrecip']] = \
    [weatherLookup(*a) for a in tuple(zip(test['latitude'],test['longitude'],test['crash_date']))]

In [12]:
import pandas as pd

In [30]:
w_list = []
for i in range(10):
    w_list.append(weather)

In [31]:
w_df = pd.DataFrame.from_records(w_list)

In [33]:
w_df.head(3)

,queryCost,latitude,longitude,resolvedAddress,address,timezone,tzoffset,days,stations,currentConditions
0,1,39.711714,-90.191958,"39.711714,-90.191958","39.711714,-90.191958",America/Chicago,-6.0,"[{'datetime': '2020-12-15', 'datetimeEpoch': 1...","{'74466653944': {'distance': 8545.0, 'latitude...","{'datetime': '14:00:00', 'datetimeEpoch': 1608..."
1,1,39.711714,-90.191958,"39.711714,-90.191958","39.711714,-90.191958",America/Chicago,-6.0,"[{'datetime': '2020-12-15', 'datetimeEpoch': 1...","{'74466653944': {'distance': 8545.0, 'latitude...","{'datetime': '14:00:00', 'datetimeEpoch': 1608..."
2,1,39.711714,-90.191958,"39.711714,-90.191958","39.711714,-90.191958",America/Chicago,-6.0,"[{'datetime': '2020-12-15', 'datetimeEpoch': 1...","{'74466653944': {'distance': 8545.0, 'latitude...","{'datetime': '14:00:00', 'datetimeEpoch': 1608..."


In [38]:
w2 = weather['currentConditions']


'14:00:00'

In [42]:
w2['date'] = weather['days'][0]['datetime']
w2['time'] = w2['datetime']
w2

{'datetime': '14:00:00',
 'datetimeEpoch': 1608062400,
 'temp': 30.9,
 'feelslike': 22.3,
 'humidity': 77.1,
 'dew': 24.6,
 'precip': 0.0,
 'precipprob': 0.0,
 'snow': 0.0,
 'snowdepth': 0.0,
 'preciptype': None,
 'windgust': None,
 'windspeed': 10.2,
 'winddir': 60.0,
 'pressure': 1023.9,
 'visibility': 9.9,
 'cloudcover': 99.7,
 'solarradiation': 99.0,
 'solarenergy': 0.4,
 'uvindex': 1.0,
 'conditions': 'Overcast',
 'icon': 'cloudy',
 'stations': ['74466653944',
  'KSPI',
  'KPPQ',
  '74466353950',
  '72439093822',
  'KIJX'],
 'source': 'obs',
 'sunrise': '07:15:05',
 'sunriseEpoch': 1608038105,
 'sunset': '16:37:23',
 'sunsetEpoch': 1608071843,
 'moonphase': 0.01,
 'date': '2020-12-15',
 'time': '14:00:00'}